# MedScrub + Claude API Integration

**No more copy/paste! Ask Claude about healthcare data directly from Jupyter.**

This notebook demonstrates:
1. ✅ Direct Claude API calls from Jupyter (no Claude Desktop needed)
2. ✅ Automatic PHI de-identification before sending to Claude
3. ✅ Automatic re-identification of responses
4. ✅ Zero copy/paste friction

## Prerequisites

1. **MedScrub JWT Token**: Get from [medscrub.dev/demo](https://medscrub.dev/demo)
2. **Claude API Key**: Get from [console.anthropic.com](https://console.anthropic.com)

## Installation

In [ ]:
# Install required packages
!pip install requests anthropic

## Download MedScrub Python Clients

**⚠️ Important:** Skip this step if you have the MedScrub repository cloned locally. The download is only needed if you're running this notebook standalone (e.g., in Google Colab or a fresh Jupyter environment) without access to the local Python client files.

**When to run this:**
- ✅ Running in Google Colab / cloud notebooks
- ✅ Standalone notebook without MedScrub repo cloned
- ✅ First-time setup on a new machine

**When to skip this:**
- ❌ You've already cloned the MedScrub repository
- ❌ The Python client files exist in your local directory
- ❌ You see "File exists" errors when running the download

In [ ]:
# Download MedScrub Python clients
# ⚠️ SKIP THIS CELL if you have the MedScrub repo cloned locally!
# Only run if you don't have medscrub_client.py and medscrub_claude.py

import urllib.request

base_url = "https://raw.githubusercontent.com/medscrub/medscrub/main/samples/python/"

urllib.request.urlretrieve(f"{base_url}medscrub_client.py", "medscrub_client.py")
urllib.request.urlretrieve(f"{base_url}medscrub_claude.py", "medscrub_claude.py")

print("✅ Downloaded MedScrub clients")

## Configure Credentials

In [ ]:
# Set your credentials here
MEDSCRUB_JWT_TOKEN = "your-jwt-token-here"  # Get from medscrub.dev/demo
CLAUDE_API_KEY = "your-claude-api-key"     # Get from console.anthropic.com

# Or use environment variables
import os
MEDSCRUB_JWT_TOKEN = os.getenv("MEDSCRUB_JWT_TOKEN", MEDSCRUB_JWT_TOKEN)
CLAUDE_API_KEY = os.getenv("ANTHROPIC_API_KEY", CLAUDE_API_KEY)

## Initialize Client

In [ ]:
from medscrub_claude import MedScrubClaude

client = MedScrubClaude(
    medscrub_jwt=MEDSCRUB_JWT_TOKEN,
    claude_api_key=CLAUDE_API_KEY
)

print("✅ MedScrub + Claude client initialized")

## Example 1: Simple Question About Patient Data

In [ ]:
# Sample patient with PHI
patient = {
    "resourceType": "Patient",
    "name": [{
        "family": "Smith",
        "given": ["John", "Michael"]
    }],
    "birthDate": "1985-03-15",
    "gender": "male",
    "telecom": [{
        "system": "email",
        "value": "john.smith@example.com"
    }]
}

# Ask Claude a question - PHI automatically scrubbed!
result = client.ask_about_fhir(
    resource=patient,
    question="What is this patient's name and date of birth?"
)

print("\n📋 Question: What is this patient's name and date of birth?\n")
print("✅ Claude's Answer:")
print(result['answer'])
print(f"\n📊 Tokens used: {result['usage']['totalTokens']}")
print(f"⏱️  Processing time: {result['processingTime']}ms")

### How It Works: Behind the Scenes

Let's see what Claude actually received (de-identified data):

In [ ]:
print("🔒 What Claude received (de-identified):")
print(result['deidentifiedAnswer'])
print("\n✨ Notice: All PHI was replaced with tokens like [FHIR_NAME_abc123]")
print("✨ Claude never saw the original PHI!")
print("\n✅ Response was automatically re-identified before returning to you")

## Example 2: Medical Analysis with FHIR Bundle

**⚠️ Important FHIR Structure Note:**

The Patient resource only contains demographics (name, DOB, gender, address). Medical conditions and medications are **separate FHIR resources** that must be sent as a Bundle. This example demonstrates the correct way to include clinical data.

In [ ]:
# FHIR Bundle with Patient + Conditions + Medications
# ✅ Correct FHIR structure: separate resources in a Bundle
patient_with_conditions = {
    "resourceType": "Bundle",
    "type": "collection",
    "entry": [
        {
            "resource": {
                "resourceType": "Patient",
                "id": "patient-emily-johnson",
                "name": [{"family": "Johnson", "given": ["Emily"]}],
                "birthDate": "1978-08-22",
                "gender": "female"
            }
        },
        {
            "resource": {
                "resourceType": "Condition",
                "id": "condition-diabetes",
                "subject": {"reference": "Patient/patient-emily-johnson"},
                "code": {
                    "coding": [{
                        "system": "http://snomed.info/sct",
                        "code": "44054006",
                        "display": "Type 2 Diabetes Mellitus"
                    }]
                },
                "onsetDateTime": "2015-03-10"
            }
        },
        {
            "resource": {
                "resourceType": "Condition",
                "id": "condition-hypertension",
                "subject": {"reference": "Patient/patient-emily-johnson"},
                "code": {
                    "coding": [{
                        "system": "http://snomed.info/sct",
                        "code": "38341003",
                        "display": "Hypertension"
                    }]
                },
                "onsetDateTime": "2012-11-05"
            }
        },
        {
            "resource": {
                "resourceType": "MedicationRequest",
                "id": "med-metformin",
                "subject": {"reference": "Patient/patient-emily-johnson"},
                "medicationCodeableConcept": {
                    "text": "Metformin 500mg"
                },
                "dosageInstruction": [{
                    "text": "Take 1 tablet twice daily with meals"
                }]
            }
        },
        {
            "resource": {
                "resourceType": "MedicationRequest",
                "id": "med-lisinopril",
                "subject": {"reference": "Patient/patient-emily-johnson"},
                "medicationCodeableConcept": {
                    "text": "Lisinopril 10mg"
                },
                "dosageInstruction": [{
                    "text": "Take 1 tablet once daily"
                }]
            }
        }
    ]
}

# Ask for medical analysis
result = client.ask_about_fhir(
    resource=patient_with_conditions,
    question="""Summarize this patient's medical conditions and current medications. 
    Are there any potential drug interactions or concerns?""",
    max_tokens=2048
)

print("\n🏥 Medical Analysis\n")
print("=" * 60)
print(result['answer'])
print("=" * 60)
print(f"\n📊 Tokens: {result['usage']['totalTokens']} | ⏱️  Time: {result['processingTime']}ms")

## Example 3: Multi-Turn Conversation

This example demonstrates maintaining context across multiple questions using the same FHIR Bundle from Example 2.

In [ ]:
# Start a conversation
result1 = client.ask_about_fhir(
    resource=patient_with_conditions,
    question="What conditions does this patient have?"
)

print("👤 User: What conditions does this patient have?")
print(f"🤖 Claude: {result1['answer']}\n")

# Continue conversation with same session
session_id = result1['sessionId']

# Follow-up question
conversation = [
    {"role": "user", "content": "What conditions does this patient have?"},
    {"role": "assistant", "content": result1['deidentifiedAnswer']},
    {"role": "user", "content": "What lifestyle changes would you recommend?"}
]

result2 = client.chat_about_fhir(
    resource=patient_with_conditions,
    messages=conversation,
    session_id=session_id
)

print("👤 User: What lifestyle changes would you recommend?")
print(f"🤖 Claude: {result2['answer']}")

## Example 4: Analyzing FHIR Bundles

In [ ]:
# FHIR Bundle with multiple resources
bundle = {
    "resourceType": "Bundle",
    "type": "collection",
    "entry": [
        {
            "resource": {
                "resourceType": "Patient",
                "name": [{"family": "Davis", "given": ["Robert"]}],
                "birthDate": "1965-05-12"
            }
        },
        {
            "resource": {
                "resourceType": "Observation",
                "code": {
                    "coding": [{
                        "display": "Blood Pressure"
                    }]
                },
                "valueQuantity": {
                    "value": 145,
                    "unit": "mmHg",
                    "system": "http://unitsofmeasure.org",
                    "code": "mm[Hg]"
                },
                "effectiveDateTime": "2024-01-15"
            }
        },
        {
            "resource": {
                "resourceType": "Observation",
                "code": {
                    "coding": [{
                        "display": "HbA1c"
                    }]
                },
                "valueQuantity": {
                    "value": 7.2,
                    "unit": "%"
                },
                "effectiveDateTime": "2024-01-15"
            }
        }
    ]
}

result = client.analyze_fhir_bundle(
    bundle=bundle,
    analysis_prompt="""Analyze this patient's recent lab results. 
    Are there any concerning trends? What follow-up would you recommend?"""
)

print("\n📊 Bundle Analysis\n")
print("=" * 60)
print(result['answer'])
print("=" * 60)

## Cleanup

Clean up session and PHI mappings when done:

In [ ]:
# Delete session to clear PHI mappings
client.cleanup()
print("✅ Session cleaned up, PHI mappings deleted")

## Using Context Manager (Recommended)

For automatic cleanup:

In [ ]:
with MedScrubClaude(medscrub_jwt=MEDSCRUB_JWT_TOKEN, claude_api_key=CLAUDE_API_KEY) as client:
    result = client.ask_about_fhir(
        resource=patient,
        question="What is this patient's gender?"
    )
    print(result['answer'])
    # Session automatically cleaned up when block exits

print("\n✅ Session automatically cleaned up!")

## Comparison: Before vs After

### ❌ Before (Copy/Paste Workflow):

1. Open Claude Desktop
2. Ask Claude to de-identify data using MCP
3. Copy de-identified output
4. Paste into Jupyter
5. **PROBLEM**: Python syntax errors!
6. Manually fix format issues
7. Run analysis

### ✅ After (Direct API Integration):

```python
result = client.ask_about_fhir(resource=patient, question="...")
print(result['answer'])
```

**That's it!** PHI automatically scrubbed, response automatically restored.

## Key Benefits

1. ✅ **Zero Copy/Paste**: Everything programmatic
2. ✅ **No Syntax Errors**: No manual formatting needed
3. ✅ **Automatic PHI Scrubbing**: MedScrub handles de-identification
4. ✅ **Context Preserved**: Responses automatically re-identified
5. ✅ **Jupyter-Native**: Works directly in notebooks
6. ✅ **HIPAA Compliant**: PHI never sent to Claude API

## Next Steps

- Try with your own FHIR data
- Integrate into your healthcare data science workflow
- Use with Synthea FHIR for realistic testing
- Build healthcare AI applications with confidence

## Resources

- **MedScrub Documentation**: [docs.medscrub.dev](https://docs.medscrub.dev)
- **Get JWT Token**: [medscrub.dev/demo](https://medscrub.dev/demo)
- **Claude API**: [console.anthropic.com](https://console.anthropic.com)
- **Python Samples**: [github.com/medscrub/medscrub/samples/python](https://github.com/medscrub/medscrub/tree/main/samples/python)

---

**Built with ❤️ for Healthcare AI Development**